# Set up Model Predictive Control Using Do-MPC Package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import casadi as cas
import do_mpc

from cas_models.continuous_time.models import StateSpaceModelCT
from cas_models.transformations import connect_systems
from feed_conc_ctrl.models import MixingTankModelCT, RatioControlledFlowMixerCT

## Example: Two-Tank Feed Blending System

```none
        ┌────────┐        
  ─────►┤ Tank 1 │        
        │        │        
        │        ├──┐     
        └────────┘  │     
                    │     
                    │     
                    ├────►
        ┌────────┐  │     
  ─────►┤ Tank 2 │  │     
        │        │  │     
        │        ├──┘     
        └────────┘        
```

States
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass

Inputs:
 1. Tank 1 inflow rate
 2. Tank 1 inflow concentration
 3. Tank 1 outflow rate
 4. Tank 2 inflow rate
 5. Tank 2 inflow concentration
 6. Tank 2 outflow rate

Outputs:
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 1 outflow concentration
 4. Tank 2 level
 5. Tank 2 mass
 6. Tank 2 outflow concentration
 7. Mixer outflow rate
 8. Mixer outflow concentration

In [3]:
def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

## Construct Mixing Tank System Model using Casadi-Models Package

In [ ]:
# Test new ratio controller
mixer = RatioControlledFlowMixerCT(2)
assert (
    str(mixer) == (
        "RatioControlledFlowMixerCT("
        "f=Function(f:(t,x[0],u[4])->(rhs[0]) SXFunction), "
        "h=Function(h:(t,x[0],u[4])->(y[3]) SXFunction), "
        "n=0, nu=4, ny=3, params={}, name='FlowMixerModel', "
        "input_names=['r_1', 'conc_in_1', 'v_dot_out', 'conc_in_2'], "
        "state_names=[], "
        "output_names=['v_dot_in_1', 'v_dot_in_2', 'conc_out'])")
)

t = 0.0
x = []

# Equal inflow concentrations, 50:50 ratio
u = [0.5, 10.0, 2.0, 10.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [1, 1, 10])

# Equal inflow concentrations, 20:80 ratio
u = [0.2, 10.0, 2.0, 10.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [0.4, 1.6, 10])

# Different concentrations, 50:50 ratio
u = [0.5, 10.0, 2.0, 0.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [1, 1, 5])

#  Mixer wit 3 inputs
mixer = RatioControlledFlowMixerCT(3)
assert (
    str(mixer) == (
        "RatioControlledFlowMixerCT("
        "f=Function(f:(t,x[0],u[6])->(rhs[0]) SXFunction), "
        "h=Function(h:(t,x[0],u[6])->(y[4]) SXFunction), "
        "n=0, nu=6, ny=4, params={}, name='FlowMixerModel', "
        "input_names=['r_1', 'conc_in_1', 'r_2', 'conc_in_2', 'v_dot_out', 'conc_in_3'], "
        "state_names=[], "
        "output_names=['v_dot_in_1', 'v_dot_in_2', 'v_dot_in_3', 'conc_out'])")
)

u = [0.3, 10.0, 0.4, 20.0, 2.0, 30.0]
y = mixer.h(t, x, u)
assert np.allclose(
    np.array(y).flatten(), 
    [0.3 * 2, 0.4 * 2, 0.3 * 2, (0.3 * 10 + 0.4 * 20 + 0.3 * 30)]
)

In [ ]:
D = 5  # tank diameter [m]
n_tanks = 2
tank_names = [f"tank_{i+1}" for i in range(n_tanks)]

# Initialize tank system models
systems = [MixingTankModelCT(D=D, name=name) for name in tank_names]

# Add a flow mixer to join flows from two tanks
systems.append(RatioControlledFlowMixerCT(2, name="mixer"))

# Check inputs and outputs
for sys in systems:
    print_sys_dimensions(sys)

In [ ]:
# Connect all systems together
connections = {
    'mixer_conc_in_1': 'tank_1_conc_out',
    'mixer_conc_in_2': 'tank_2_conc_out',
    'tank_1_v_dot_out': 'mixer_v_dot_in_1',
    'tank_2_v_dot_out': 'mixer_v_dot_in_2',
}

model_class = StateSpaceModelCT
feed_tanks_system = connect_systems(
    systems, connections, model_class, name="tank_system_21", verbose_names=True
) 
print_sys_dimensions(feed_tanks_system)

## Control Model Design

**States**
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass

**Manipulated Variables (MVs)**
 1. Tank 1 inflow rate
 2. Tank 2 inflow rate
 3. Mixer inflow ratio

where mixer inflow ratio = tank 1 outflow rate / 
    (tank 1 outflow rate + tank 2 outflow rate)

**Unmeasured Disturbances**
 1. Tank 1 inflow concentration
 2. Tank 2 inflow concentration
 3. Mixer outflow rate

**Controlled Variables (CVs)**
 1. Tank 1 level
 2. Tank 1 outflow concentration
 3. Tank 2 level
 4. Tank 2 outflow concentration
 5. Mixer outflow concentration



In [ ]:
control_design = {
    "state_names": ['tank_1_L', 'tank_1_m', 'tank_2_L', 'tank_2_m'],
    "mv_names": ['tank_1_v_dot_in', 'tank_2_v_dot_in', 'mixer_r_1']
}

## Construct Do-MPC System Model

In [ ]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

# Define state variables
for tank_name in tank_names:
    model.set_variable(var_type='_x', var_name=tank_name + '_L', shape=(1, 1))
    model.set_variable(var_type='_x', var_name=tank_name + '_m', shape=(1, 1))
n = model.x.shape[0]

# Augment model with states for unmeasured disturbances
for tank_name in tank_names:
    model.set_variable(var_type='_x', var_name=tank_name + '_conc_in', shape=(1, 1))
model.set_variable(var_type='_x', var_name='mixer_v_dot_out', shape=(1, 1))
n_aug = model.x.shape[0]

print("States: ", list(model.x.keys()))

In [ ]:
# Define manipulated input variables
for tank_name in tank_names:
    model.set_variable(var_type='_u', var_name=tank_name + '_v_dot_in', shape=(1, 1))
model.set_variable(var_type='_u', var_name='mixer_ratio', shape=(1, 1))

print("MVs: ", list(model.u.keys()))

In [ ]:
# Define experessions for tank outflow rates based on mixer ratio 
tank_1_v_dot_out = model.u['mixer_ratio'] * model.x['mixer_v_dot_out']
tank_2_v_dot_out = (1.0 - model.u['mixer_ratio']) * model.x['mixer_v_dot_out']

t = 0.0  # not used in this model
states = cas.vcat(model.x[feed_tanks_system.state_names])
inputs = cas.vcat([
    model.u['tank_1_v_dot_in'], 
    model.x['tank_1_conc_in'], 
    tank_1_v_dot_out, 
    model.u['tank_2_v_dot_in'], 
    model.x['tank_2_conc_in'], 
    tank_2_v_dot_out
])
assert inputs.shape == (feed_tanks_system.nu, 1)

# Create expressions for righthand side of ODE
rhs = feed_tanks_system.f(t, states, inputs)
rhs_expr = {
    name: rhs[i] for i, name in enumerate(feed_tanks_system.state_names)
}
print(f"ODE rhs expressions:")
for name, expr in rhs_expr.items():
    print(f"{name:>18s}: {expr}")

In [ ]:
# Set rhs expressions in Do-MPC model
model.x.keys()

In [ ]:
inputs_dict

In [ ]:
# Define manipulated input variables
for tank_name in tank_names:
    var_name = tank_name + '_v_dot_in'
    model.set_rhs(var_name, inputs[var_name])
var_name='mixer_ratio'
model.set_rhs(var_name, rhs_expr[var_name])

In [ ]:
# Create expressions for all outputs
outputs = feed_tanks_system.h(t, states, inputs)
output_expr = {
    name: outputs[i] for i, name in enumerate(feed_tanks_system.output_names)
}
print(f"Output expressions:")
for name, expr in output_expr.items():
    print(f"{name:>18s}: {expr}")

In [ ]:
# Define measured variables and output expressions
for tank_name in tank_names:
    meas_name = tank_name + '_L'
    model.set_meas(meas_name=meas_name, expr=output_expr[meas_name])
    meas_name = tank_name + '_conc_out'
    model.set_meas(meas_name=meas_name, expr=output_expr[meas_name])
meas_name = 'mixer_conc_out'
model.set_meas(
    meas_name=meas_name, expr=output_expr[meas_name]
)
print("CVs: ", list(model.y.keys()))